In [1]:
import torch
from matplotlib import pyplot as plt
from PIL import Image

from models import Encoder, Decoder

from pathlib import Path
from get_loader import MSCOCODataset, get_loader

from data_prep_utils import *
from utils import train, save_model, load_model, plot_loss, predict
import json
import numpy as np
import time

In [2]:
# for building vocab
FREQ_THRESHOLD = 5

# for data loader
BATCH_SIZE = 32
CAPS_PER_IMAGE = 5 # how many captions for each image to include in data set

# for encoder and decoder
EMBED_SIZE = 512 # dimension of vocab embedding vector
HIDDEN_SIZE = 512
NUM_LAYERS = 1 #hidden layers in LTSM

# optimiser parameters
OPT_PARAMS = {'lr':1e-3, 'weight_decay':1e-3, 'betas':(0.9, 0.999), 'eps':1e-08}

# checkpoint file
CHECKPOINT = '../model/image_captioning_model_v10.pth'
PRINT_EVERY = 300 # run print_every batches and then
# print running results. For bigger batch size make this 
# number smaller if you want to see regular output

In [3]:
root = Path('../Datasets/coco')
imgs_path = root/'images'/'train2017'
imgs_path_test = root/'images'/'val2017'

# prepare_datasets(train_percent = 0.87, super_categories=['sports'],
#                     max_train=15000, max_val=3000, max_test=3000)

train_captions_path = root/'annotations'/'sports_captions_train.json'
val_captions_path = root/'annotations'/'sports_captions_val.json'
test_captions_path = root/'annotations'/'sports_captions_test.json'

#### build vocab using full original coco train
# build_vocab(freq_threshold=FREQ_THRESHOLD)

# load vocab
with open('../vocabulary/string_to_index.json') as json_file:
    word2idx = json.load(json_file)
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
test_loader_params = {
    'images_path': imgs_path_test,
    'captions_path': test_captions_path,
    'freq_threshold': FREQ_THRESHOLD,
    'caps_per_image': 1,
    'mode': "train",
    'transform': None,
    'batch_size': 1,
    'shuffle': True,
    'word2idx': word2idx,
}

test_dl, test_dataset = get_loader(**test_loader_params)

vocab_size = len(test_dataset.word2idx)

encoder = Encoder(embed_size=EMBED_SIZE, pretrained=True, model_weight_path="../model/resnet152_model.pth")
decoder = Decoder(embed_size=EMBED_SIZE, hidden_size=HIDDEN_SIZE, vocab_size=vocab_size, num_layers=NUM_LAYERS)

In [5]:
if Path(CHECKPOINT).exists():
    encoder, decoder, training_loss, validation_loss, hyper_params = load_model(encoder, decoder, CHECKPOINT)
else:
    print(f'{CHECKPOINT} file does not exist, training startging from scratch')
    training_loss = None
    validation_loss = None

In [6]:
idx, im, cap, capt_lens = next(iter(test_dl))
im.to(device)
cap.to(device)

encoder.eval()
decoder.eval()

features = encoder(im)
caption = decoder.predict(features, max_length=40,
                           idx2word=test_dataset.idx2word)
print('predicted caption for image {}:'.format(idx[0]))
print(' '.join([test_dataset.idx2word[idx] 
                for idx in caption]))

image = test_dataset.coco.get_img(idx[0])
# plt.imshow(torch.tensor(image).permute(1,2,0));
image

TypeError: predict() got an unexpected keyword argument 'max_length'

In [ ]:
plot_loss(training_loss, validation_loss)

In [ ]:
len(test_dataset.idx2word)

In [ ]:
predict(encoder, decoder, image, test_dataset.idx2word, test_dataset.word2idx, device)

In [27]:
root = Path('../Datasets/coco')
imgs_path = root/'images'/'train2017'
imgs_path_test = root/'images'/'val2017'

# prepare_datasets(train_percent = 0.87, super_categories=['sports'],
#                     max_train=15000, max_val=3000, max_test=3000)

train_captions_path = root/'annotations'/'sports_captions_train.json'
val_captions_path = root/'annotations'/'sports_captions_val.json'
test_captions_path = root/'annotations'/'sports_captions_test.json'

#### build vocab using full original coco train
build_vocab(freq_threshold=FREQ_THRESHOLD,
            captions_file=CAPTIONS_FILE)

# load vocab
with open('../vocabulary/string_to_index.json') as json_file:
    word2idx = json.load(json_file)

NameError: name 'CAPTIONS_FILE' is not defined

In [28]:
train_loader_params = {
    'images_path': imgs_path,
    'captions_path': train_captions_path,
    'freq_threshold': FREQ_THRESHOLD,
    'caps_per_image': CAPS_PER_IMAGE,
    'mode': "train",
    'transform': None,
    'batch_size': BATCH_SIZE,
    'shuffle': True,
    'word2idx': word2idx,
}

val_loader_params = {
    'images_path': imgs_path,
    'captions_path': val_captions_path,
    'freq_threshold': FREQ_THRESHOLD,
    'caps_per_image': 1,
    'mode': "train",
    'transform': None,
    'batch_size': BATCH_SIZE,
    'shuffle': True,
    'word2idx': word2idx,
}

train_dl, train_dataset = get_loader(**train_loader_params)
val_dl, val_dataset = get_loader(**val_loader_params)

vocab_size = len(train_dataset.word2idx)

In [29]:
len(train_dataset.img_deque)

75000

In [31]:
len(set([d[0] for d in train_dataset.img_deque]))

15000

In [61]:
batch = next(iter(train_dl))
dir(zip(*batch))
len(zip(*batch))

TypeError: object of type 'zip' has no len()

In [56]:
for a, b, c in zip(*batch):
    print(a)

26619
74975
30938
69524
48167
62258
66599
11675
48735
52131
66514
14908
51218
62562
9990
67815
417
16824
22131
34759
38040
53261
54655
50356
6691
53155
40020
71626
16588
17586
26323
23428
